6年分のデータを使ってKfoldを実装していく

In [1]:
!pip install lightgbm -q

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [3]:
#df = pd.read_csv("../data/process-data/train4-修正済み2.csv")
# 異常値修正後
df = pd.read_csv("../data/process-data/train4-修正済み3.csv")

# 水位の特徴量


df_water1 = pd.read_csv("../data/process-data/water-process-label1-v1-修正済み3.csv")
df_water0 = pd.read_csv("../data/process-data/water-process-label0-v1-修正済み3.csv")

# 潮位ラベルや水位観測所に対応した雨量観測所の情報をまとめたもの
df_label = pd.read_excel('../data/process-data/雨量観測所対応and潮位ラベル.xlsx')


# 雨量追加フラグ
if True:
    # 潮位ラベル結合
    df_water1 = pd.merge(df_water1, df_label, left_on="station", right_on="水位観測所", how="left")
    df_water1 = df_water1.drop('水位観測所',axis=1)
    df_water0 = pd.merge(df_water0, df_label, left_on="station", right_on="水位観測所", how="left")
    df_water0 = df_water0.drop('水位観測所',axis=1)
    
    
    df_rain = pd.read_csv("../data/process-data/rain-process-easy-per3h.csv")
    
    d = {}
    for column in df_rain.columns:
        if column not in ["date", "city"]:
            d[column] = column+"_rain"
    df_rain = df_rain.rename(d,axis=1)
    
    df_water1 = pd.merge(df_water1, df_rain, left_on=["date", "使用する雨量観測所"], right_on=["date", "station_rain"], how="left")
    df_water1 = df_water1.drop(columns=['使用する雨量観測所', "潮位ラベル", "station_rain","city"])
    df_water0 = pd.merge(df_water0, df_rain, left_on=["date", "使用する雨量観測所"], right_on=["date", "station_rain"], how="left")
    df_water0 = df_water0.drop(columns=['使用する雨量観測所', "潮位ラベル", "station_rain","city"])


# カテゴリ化
le = LabelEncoder()
le.fit(df["station"])
# array([43, 43, 43, ...,  1,  1,  1])
print(le.transform(df["station"]))
df["station_c"] = le.transform(df["station"])
# np.save('../classes-station-166-v2.npy', le.classes_)    

# これを学習用に使う
df = df.drop(columns=["water level"])
df1 = df[df["station"].isin(list(df_label[df_label["潮位ラベル"]==1]["水位観測所"]))]
df0 = df[df["station"].isin(list(df_label[df_label["潮位ラベル"]==0]["水位観測所"]))]
df1 = pd.merge(df1, df_water1, on=["date", "station", "river"], how="left")
df0 = pd.merge(df0, df_water0, on=["date", "station", "river"], how="left")

[43 43 43 ...  1  1  1]


```
0~364,365~729,730~1094,1095~1460,1461~1825,1826~2190で、前半5つの内4つを学習、残りを検証用に回す  
k=0は4.1で作ったモデルとする
```

In [4]:
def make_fold(x):
    if x <= 364:
        return 4
    elif x <= 729:
        return 3
    elif x <= 1094:
        return 2
    elif x <= 1460:
        return 1
    elif x <= 1825:
        return 0
    else:
        return -1
df1["fold"] = df1["date"].apply(lambda x : make_fold(x))
df0["fold"] = df0["date"].apply(lambda x : make_fold(x))
df1["fold"].value_counts()

 0    80957
 1    65198
 2    60116
 4    59849
-1    59636
 3    46620
Name: fold, dtype: int64

### 潮位ラベル1

In [6]:
train_col = []
categorical_col = ["station_c"]
for col in df1.columns:
    if col not in ["date", "station", "river", "hour", "next water level","fold"]:
            train_col.append(col)
else:
    print(train_col)

['hour_sin', 'hour_cos', 'station_c', 'h-0', 'h-1', 'h-2', 'h-3', 'h-4', 'h-5', 'h-6', 'h-7', 'h-8', 'h-9', 'h-10', 'h-11', 'h-12', 'h-13', 'h-14', 'h-15', 'h-16', 'h-17', 'h-18', 'h-19', 'h-20', 'h-21', 'h-22', 'h-23', 'mean', 'max', 'min', 'std', 'nullcount', '0-12mean', '12-24mean', 'h-0_1d_ago', 'h-1_1d_ago', 'h-2_1d_ago', 'h-3_1d_ago', 'h-4_1d_ago', 'h-5_1d_ago', 'h-6_1d_ago', 'h-7_1d_ago', 'h-8_1d_ago', 'h-9_1d_ago', 'h-10_1d_ago', 'h-11_1d_ago', 'h-12_1d_ago', 'h-13_1d_ago', 'h-14_1d_ago', 'h-15_1d_ago', 'h-16_1d_ago', 'h-17_1d_ago', 'h-18_1d_ago', 'h-19_1d_ago', 'h-20_1d_ago', 'h-21_1d_ago', 'h-22_1d_ago', 'h-23_1d_ago', 'mean_1d_ago', 'max_1d_ago', 'min_1d_ago', 'std_1d_ago', 'nullcount_1d_ago', '0-12mean_1d_ago', '12-24mean_1d_ago', 'mean_2d_ago', 'max_2d_ago', 'min_2d_ago', 'std_2d_ago', 'nullcount_2d_ago', '0-12mean_2d_ago', '12-24mean_2d_ago', 'mean_3d_ago', 'max_3d_ago', 'min_3d_ago', 'std_3d_ago', 'nullcount_3d_ago', '0-12mean_3d_ago', '12-24mean_3d_ago', 'mean_4d_ago', 

#### k=-1
作成済み

#### k=0

In [7]:
fold=0
# 5年、1年
df_train_x = df1[df1["fold"]!=fold][train_col]
df_valid_x = df1[df1["fold"]==fold][train_col]

df_train_y = df1[df1["fold"]!=fold]["next water level"]
df_valid_y = df1[df1["fold"]==fold]["next water level"]

print(df_train_x.shape)
print(df_valid_x.shape)

(291419, 100)
(80957, 100)


In [8]:
# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 30000, early_stopping_rounds = 100, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 500)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20933
[LightGBM] [Info] Number of data points in the train set: 291419, number of used features: 100
[LightGBM] [Info] Start training from score 0.531517
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 0.174228
[1000]	valid_0's rmse: 0.164786
[1500]	valid_0's rmse: 0.160233
[2000]	valid_0's rmse: 0.157762
[2500]	valid_0's rmse: 0.156243
[3000]	valid_0's rmse: 0.155016
[3500]	valid_0's rmse: 0.154368
[4000]	valid_0's rmse: 0.153842
[4500]	valid_0's rmse: 0.153439
[5000]	valid_0's rmse: 0.152927
[5500]	valid_0's rmse: 0.152563
[6000]	valid_0's rmse: 0.152197
[6500]	valid_0's rmse: 0.151952
[7000]	valid_0's rmse: 0.151676
[7500]	valid_0's rmse: 0.15142
Early stopping, best iteration is:
[7796]	valid_

In [11]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.15127995205875355


In [12]:
importance = model.feature_importance()
# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 28097
hour_cos: 23191
station_c: 2059
h-0: 2791
h-1: 2294
h-2: 2416
h-3: 2069
h-4: 2116
h-5: 2761
h-6: 1998
h-7: 2151
h-8: 2148
h-9: 2346
h-10: 2720
h-11: 2923
h-12: 2684
h-13: 2538
h-14: 2493
h-15: 2291
h-16: 2356
h-17: 2493
h-18: 2486
h-19: 2308
h-20: 2481
h-21: 2245
h-22: 2386
h-23: 3004
mean: 3658
max: 2503
min: 1707
std: 2348
nullcount: 163
0-12mean: 1952
12-24mean: 2394
h-0_1d_ago: 2299
h-1_1d_ago: 1777
h-2_1d_ago: 1640
h-3_1d_ago: 1511
h-4_1d_ago: 1516
h-5_1d_ago: 1537
h-6_1d_ago: 1563
h-7_1d_ago: 1677
h-8_1d_ago: 1698
h-9_1d_ago: 1805
h-10_1d_ago: 2117
h-11_1d_ago: 2105
h-12_1d_ago: 2056
h-13_1d_ago: 2266
h-14_1d_ago: 1959
h-15_1d_ago: 2096
h-16_1d_ago: 1954
h-17_1d_ago: 1934
h-18_1d_ago: 1773
h-19_1d_ago: 1686
h-20_1d_ago: 1707
h-21_1d_ago: 1555
h-22_1d_ago: 1679
h-23_1d_ago: 2408
mean_1d_ago: 1437
max_1d_ago: 1783
min_1d_ago: 1291
std_1d_ago: 1939
nullcount_1d_ago: 141
0-12mean_1d_ago: 1438
12-24mean_1d_ago: 1775
mean_2d_ago: 1526
max_2d_ago: 2164
min_2d_ago: 1721
s

In [13]:
import pickle

file = '../model/model_20230102-2-7-fold0.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=1

In [14]:
fold=1
# 5年、1年
df_train_x = df1[df1["fold"]!=fold][train_col]
df_valid_x = df1[df1["fold"]==fold][train_col]

df_train_y = df1[df1["fold"]!=fold]["next water level"]
df_valid_y = df1[df1["fold"]==fold]["next water level"]

print(df_train_x.shape)
print(df_valid_x.shape)

(307178, 100)
(65198, 100)


In [15]:
# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 30000, early_stopping_rounds = 100, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 500)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.075771 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20910
[LightGBM] [Info] Number of data points in the train set: 307178, number of used features: 100
[LightGBM] [Info] Start training from score 0.553540
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 0.160602
[1000]	valid_0's rmse: 0.150103
[1500]	valid_0's rmse: 0.14572
[2000]	valid_0's rmse: 0.143035
[2500]	valid_0's rmse: 0.140932
[3000]	valid_0's rmse: 0.139218
[3500]	valid_0's rmse: 0.138131
[4000]	valid_0's rmse: 0.137011
[4500]	valid_0's rmse: 0.136269
[5000]	valid_0's rmse: 0.135645
[5500]	valid_0's rmse: 0.135159
[6000]	valid_0's rmse: 0.134719
[6500]	valid_0's rmse: 0.134332
[7000]	valid_0's rmse: 0.133986
[7500]	valid_0's rmse: 0.133736
[8000]	valid_0's rmse: 0.133498
[8500]	valid_0's rmse: 0.133283
[9000]	valid_0's rmse: 0.13307
[9500]	valid_0's 

In [16]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.1316748527792899


In [17]:
importance = model.feature_importance()
# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 52109
hour_cos: 42735
station_c: 3312
h-0: 5101
h-1: 4296
h-2: 4569
h-3: 4125
h-4: 3828
h-5: 4619
h-6: 4223
h-7: 3998
h-8: 4224
h-9: 4517
h-10: 5228
h-11: 5925
h-12: 5296
h-13: 4897
h-14: 5036
h-15: 4612
h-16: 4605
h-17: 4896
h-18: 4760
h-19: 4241
h-20: 4557
h-21: 4336
h-22: 4574
h-23: 5526
mean: 7140
max: 4664
min: 3735
std: 4931
nullcount: 311
0-12mean: 3749
12-24mean: 4587
h-0_1d_ago: 3981
h-1_1d_ago: 3706
h-2_1d_ago: 3520
h-3_1d_ago: 3145
h-4_1d_ago: 3122
h-5_1d_ago: 3206
h-6_1d_ago: 3158
h-7_1d_ago: 3324
h-8_1d_ago: 3434
h-9_1d_ago: 3671
h-10_1d_ago: 4267
h-11_1d_ago: 4269
h-12_1d_ago: 4274
h-13_1d_ago: 4299
h-14_1d_ago: 4101
h-15_1d_ago: 3789
h-16_1d_ago: 3818
h-17_1d_ago: 4064
h-18_1d_ago: 3491
h-19_1d_ago: 3135
h-20_1d_ago: 3430
h-21_1d_ago: 3145
h-22_1d_ago: 3257
h-23_1d_ago: 4428
mean_1d_ago: 2981
max_1d_ago: 3621
min_1d_ago: 2630
std_1d_ago: 3964
nullcount_1d_ago: 241
0-12mean_1d_ago: 2884
12-24mean_1d_ago: 3696
mean_2d_ago: 3158
max_2d_ago: 4320
min_2d_ago: 3485
s

In [18]:
import pickle

file = '../model/model_20230102-2-7-fold1.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=2

In [19]:
fold=2
# 5年、1年
df_train_x = df1[df1["fold"]!=fold][train_col]
df_valid_x = df1[df1["fold"]==fold][train_col]

df_train_y = df1[df1["fold"]!=fold]["next water level"]
df_valid_y = df1[df1["fold"]==fold]["next water level"]

print(df_train_x.shape)
print(df_valid_x.shape)

(312260, 100)
(60116, 100)


In [20]:
# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 30000, early_stopping_rounds = 100, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 500)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062651 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20890
[LightGBM] [Info] Number of data points in the train set: 312260, number of used features: 100
[LightGBM] [Info] Start training from score 0.550600
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 0.162928
[1000]	valid_0's rmse: 0.153396
[1500]	valid_0's rmse: 0.148733
[2000]	valid_0's rmse: 0.146223
[2500]	valid_0's rmse: 0.144673
[3000]	valid_0's rmse: 0.143349
[3500]	valid_0's rmse: 0.14214
[4000]	valid_0's rmse: 0.141266
[4500]	valid_0's rmse: 0.140775
[5000]	valid_0's rmse: 0.140263
[5500]	valid_0's rmse: 0.139872
[6000]	valid_0's rmse: 0.139522
[6500]	valid_0's rmse: 0.139298
[7000]	valid_0's rmse: 0.138998
[7500]	valid_0's rmse: 0.13875
[8000]	valid_0's rmse: 0.13855
[8500]	valid_0's rmse: 0.138375
Early stopping, best iteration is:
[8791]	valid_0'

In [21]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.1382709710855723


In [22]:
importance = model.feature_importance()
# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 31306
hour_cos: 25957
station_c: 2238
h-0: 3156
h-1: 2633
h-2: 2683
h-3: 2346
h-4: 2413
h-5: 2985
h-6: 2503
h-7: 2385
h-8: 2473
h-9: 2697
h-10: 3032
h-11: 3397
h-12: 3080
h-13: 2878
h-14: 2693
h-15: 2644
h-16: 2554
h-17: 2961
h-18: 2850
h-19: 2601
h-20: 2757
h-21: 2588
h-22: 2808
h-23: 3430
mean: 4513
max: 2687
min: 2025
std: 2789
nullcount: 226
0-12mean: 2199
12-24mean: 2547
h-0_1d_ago: 2271
h-1_1d_ago: 2028
h-2_1d_ago: 1995
h-3_1d_ago: 1783
h-4_1d_ago: 1838
h-5_1d_ago: 1778
h-6_1d_ago: 1796
h-7_1d_ago: 1851
h-8_1d_ago: 1960
h-9_1d_ago: 2037
h-10_1d_ago: 2388
h-11_1d_ago: 2519
h-12_1d_ago: 2377
h-13_1d_ago: 2396
h-14_1d_ago: 2166
h-15_1d_ago: 2183
h-16_1d_ago: 2132
h-17_1d_ago: 2152
h-18_1d_ago: 1894
h-19_1d_ago: 1804
h-20_1d_ago: 1890
h-21_1d_ago: 1828
h-22_1d_ago: 1897
h-23_1d_ago: 2667
mean_1d_ago: 1671
max_1d_ago: 1982
min_1d_ago: 1418
std_1d_ago: 2270
nullcount_1d_ago: 94
0-12mean_1d_ago: 1582
12-24mean_1d_ago: 1896
mean_2d_ago: 1783
max_2d_ago: 2451
min_2d_ago: 1805
st

In [23]:
import pickle

file = '../model/model_20230102-2-7-fold2.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=3

In [24]:
fold=3
# 5年、1年
df_train_x = df1[df1["fold"]!=fold][train_col]
df_valid_x = df1[df1["fold"]==fold][train_col]

df_train_y = df1[df1["fold"]!=fold]["next water level"]
df_valid_y = df1[df1["fold"]==fold]["next water level"]

print(df_train_x.shape)
print(df_valid_x.shape)

(325756, 100)
(46620, 100)


In [27]:
# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 30000, early_stopping_rounds = 200, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 500)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049376 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20925
[LightGBM] [Info] Number of data points in the train set: 325756, number of used features: 100
[LightGBM] [Info] Start training from score 0.562072
Training until validation scores don't improve for 200 rounds
[500]	valid_0's rmse: 0.275249
[1000]	valid_0's rmse: 0.270946
[1500]	valid_0's rmse: 0.267679
[2000]	valid_0's rmse: 0.265819
[2500]	valid_0's rmse: 0.264451
[3000]	valid_0's rmse: 0.2634
[3500]	valid_0's rmse: 0.261952
[4000]	valid_0's rmse: 0.261056
[4500]	valid_0's rmse: 0.260812
[5000]	valid_0's rmse: 0.260237
[5500]	valid_0's rmse: 0.259903
[6000]	valid_0's rmse: 0.259607
[6500]	valid_0's rmse: 0.259145
[7000]	valid_0's rmse: 0.258899
[7500]	valid_0's rmse: 0.258728
[8000]	valid_0's rmse: 0.258521
Early stopping, best iteration is:
[7917]	valid_0's rmse: 0.258437


In [28]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.258436791424447


In [29]:
importance = model.feature_importance()
# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 27907
hour_cos: 23382
station_c: 1925
h-0: 2938
h-1: 2401
h-2: 2504
h-3: 2126
h-4: 2157
h-5: 2747
h-6: 2206
h-7: 2414
h-8: 2274
h-9: 2658
h-10: 3016
h-11: 3293
h-12: 3193
h-13: 2838
h-14: 2734
h-15: 2606
h-16: 2652
h-17: 2925
h-18: 2826
h-19: 2525
h-20: 2637
h-21: 2403
h-22: 2511
h-23: 3078
mean: 4318
max: 2357
min: 1682
std: 2679
nullcount: 180
0-12mean: 1919
12-24mean: 2336
h-0_1d_ago: 1952
h-1_1d_ago: 1809
h-2_1d_ago: 1906
h-3_1d_ago: 1522
h-4_1d_ago: 1572
h-5_1d_ago: 1535
h-6_1d_ago: 1453
h-7_1d_ago: 1593
h-8_1d_ago: 1794
h-9_1d_ago: 2014
h-10_1d_ago: 2265
h-11_1d_ago: 2479
h-12_1d_ago: 2273
h-13_1d_ago: 2476
h-14_1d_ago: 2094
h-15_1d_ago: 2174
h-16_1d_ago: 1927
h-17_1d_ago: 2018
h-18_1d_ago: 1668
h-19_1d_ago: 1639
h-20_1d_ago: 1728
h-21_1d_ago: 1747
h-22_1d_ago: 1751
h-23_1d_ago: 2283
mean_1d_ago: 1399
max_1d_ago: 1870
min_1d_ago: 1178
std_1d_ago: 1911
nullcount_1d_ago: 74
0-12mean_1d_ago: 1408
12-24mean_1d_ago: 1834
mean_2d_ago: 1320
max_2d_ago: 2106
min_2d_ago: 1376
st

In [31]:
df_valid = df1[(df1["date"] >= 365) & (df1["date"] < 730)][["date", "station", "river", "hour"]]
df_valid["loss"] = (df_valid_y-y_pred)**2
df_valid

,date,station,river,hour,loss
59849,365,草津(国),太田川放水路,0,0.014983
59850,365,草津(国),太田川放水路,1,0.025589
59851,365,草津(国),太田川放水路,2,0.042389
59852,365,草津(国),太田川放水路,3,0.030581
59853,365,草津(国),太田川放水路,4,0.038986
...,...,...,...,...,...
106464,729,竹原,賀茂川,19,0.004404
106465,729,竹原,賀茂川,20,0.000141
106466,729,竹原,賀茂川,21,0.000692
106467,729,竹原,賀茂川,22,0.000058


In [32]:
df_valloss = df_valid.groupby("station")["loss"].mean()
df_valloss

station
三篠橋(国)         0.071988
岡ノ下            0.044757
江波(旧太田川)(国)    0.073743
祇園大橋(国)        0.084141
竹原             0.047376
草津(国)          0.084179
長和久(国)         0.063520
Name: loss, dtype: float64

In [33]:
import pickle

file = '../model/model_20230102-2-7-fold3.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=4

In [34]:
fold=4
# 5年、1年
df_train_x = df1[df1["fold"]!=fold][train_col]
df_valid_x = df1[df1["fold"]==fold][train_col]

df_train_y = df1[df1["fold"]!=fold]["next water level"]
df_valid_y = df1[df1["fold"]==fold]["next water level"]

print(df_train_x.shape)
print(df_valid_x.shape)

(312527, 100)
(59849, 100)


In [35]:
# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 30000, early_stopping_rounds = 100, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 500)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050105 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20854
[LightGBM] [Info] Number of data points in the train set: 312527, number of used features: 100
[LightGBM] [Info] Start training from score 0.559333
Training until validation scores don't improve for 100 rounds
[500]	valid_0's rmse: 0.150624
[1000]	valid_0's rmse: 0.139807
[1500]	valid_0's rmse: 0.135309
[2000]	valid_0's rmse: 0.132284
[2500]	valid_0's rmse: 0.130506
[3000]	valid_0's rmse: 0.129101
[3500]	valid_0's rmse: 0.127952
[4000]	valid_0's rmse: 0.127076
[4500]	valid_0's rmse: 0.126437
[5000]	valid_0's rmse: 0.12595
[5500]	valid_0's rmse: 0.125464
[6000]	valid_0's rmse: 0.125003
[6500]	valid_0's rmse: 0.124626
[7000]	valid_0's rmse: 0.124284
[7500]	valid_0's rmse: 0.123955
[8000]	valid_0's rmse: 0.123734
[8500]	valid_0's

In [36]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.12148749190504753


In [37]:
importance = model.feature_importance()
# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 65251
hour_cos: 53584
station_c: 4450
h-0: 6630
h-1: 5425
h-2: 5378
h-3: 4772
h-4: 4869
h-5: 6104
h-6: 4927
h-7: 5306
h-8: 5146
h-9: 5750
h-10: 6651
h-11: 6736
h-12: 6432
h-13: 6161
h-14: 5808
h-15: 5948
h-16: 5573
h-17: 5957
h-18: 5553
h-19: 5311
h-20: 6175
h-21: 5553
h-22: 5480
h-23: 6666
mean: 8603
max: 5881
min: 4527
std: 6104
nullcount: 348
0-12mean: 4668
12-24mean: 5041
h-0_1d_ago: 4998
h-1_1d_ago: 4433
h-2_1d_ago: 3982
h-3_1d_ago: 3722
h-4_1d_ago: 3883
h-5_1d_ago: 3929
h-6_1d_ago: 4163
h-7_1d_ago: 4319
h-8_1d_ago: 4272
h-9_1d_ago: 4549
h-10_1d_ago: 5529
h-11_1d_ago: 5285
h-12_1d_ago: 5227
h-13_1d_ago: 5267
h-14_1d_ago: 5106
h-15_1d_ago: 4939
h-16_1d_ago: 4768
h-17_1d_ago: 4717
h-18_1d_ago: 4199
h-19_1d_ago: 4055
h-20_1d_ago: 4207
h-21_1d_ago: 4043
h-22_1d_ago: 4174
h-23_1d_ago: 5281
mean_1d_ago: 3611
max_1d_ago: 4873
min_1d_ago: 3397
std_1d_ago: 5122
nullcount_1d_ago: 193
0-12mean_1d_ago: 3567
12-24mean_1d_ago: 4295
mean_2d_ago: 4061
max_2d_ago: 5364
min_2d_ago: 4392
s

In [38]:
import pickle

file = '../model/model_20230102-2-7-fold4.pkl'
pickle.dump(model, open(file, 'wb'))

### 潮位ラベル0

In [5]:
train_col = []
categorical_col = ["station_c"]
for col in df0.columns:
    if col not in ["date", "station", "river", "hour", "next water level","fold"]:
            train_col.append(col)
else:
    print(train_col)

['hour_sin', 'hour_cos', 'station_c', 'h-0', 'h-6', 'h-12', 'h-15', 'h-18', 'h-21', 'h-22', 'h-23', 'mean', 'max', 'min', 'std', 'nullcount', '0-12mean', '12-24mean', '0-6mean', '6-12mean', '12-18mean', '18-24mean', 'std_1d_ago', 'nullcount_1d_ago', '0-6mean_1d_ago', '6-12mean_1d_ago', '12-18mean_1d_ago', '18-24mean_1d_ago', 'std_2d_ago', 'nullcount_2d_ago', '0-12mean_2d_ago', '12-24mean_2d_ago', '12-14-mean_rain', '12-14-max_rain', '15-17-mean_rain', '15-17-max_rain', '18-20-mean_rain', '18-20-max_rain', '21-23-mean_rain', '21-23-max_rain', 'mean_rain', 'mean_1d_ago_rain', 'mean_2d_ago_rain']


#### k=0

In [6]:
fold=0
# 5年、1年
df_train_x = df0[df0["fold"]!=fold][train_col]
df_valid_x = df0[df0["fold"]==fold][train_col]

df_train_y = df0[df0["fold"]!=fold]["next water level"]
df_valid_y = df0[df0["fold"]==fold]["next water level"]

# validだけ変化
print(df_train_x.shape)
print(df_valid_x.shape)

(6660147, 43)
(1426943, 43)


In [7]:
import lightgbm as lgb

# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 4000, early_stopping_rounds = 200, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.561633 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8139
[LightGBM] [Info] Number of data points in the train set: 6660147, number of used features: 43
[LightGBM] [Info] Start training from score 2.397157
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.124161
[200]	valid_0's rmse: 0.123487
[300]	valid_0's rmse: 0.122902
[400]	valid_0's rmse: 0.122966
[500]	valid_0's rmse: 0.122991
Early stopping, best iteration is:
[326]	valid_0's rmse: 0.122843


In [8]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.12284334738804871


In [9]:
importance = model.feature_importance()

# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 541
hour_cos: 508
station_c: 2432
h-0: 212
h-6: 73
h-12: 98
h-15: 83
h-18: 97
h-21: 121
h-22: 159
h-23: 500
mean: 29
max: 72
min: 126
std: 466
nullcount: 29
0-12mean: 10
12-24mean: 50
0-6mean: 17
6-12mean: 27
12-18mean: 35
18-24mean: 107
std_1d_ago: 345
nullcount_1d_ago: 51
0-6mean_1d_ago: 110
6-12mean_1d_ago: 93
12-18mean_1d_ago: 66
18-24mean_1d_ago: 95
std_2d_ago: 345
nullcount_2d_ago: 42
0-12mean_2d_ago: 177
12-24mean_2d_ago: 49
12-14-mean_rain: 178
12-14-max_rain: 76
15-17-mean_rain: 183
15-17-max_rain: 97
18-20-mean_rain: 190
18-20-max_rain: 66
21-23-mean_rain: 537
21-23-max_rain: 224
mean_rain: 463
mean_1d_ago_rain: 295
mean_2d_ago_rain: 306


In [10]:
import pickle

file = '../model/model_20230102-2-159-fold0.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=1

In [11]:
fold=1
# 5年、1年
df_train_x = df0[df0["fold"]!=fold][train_col]
df_valid_x = df0[df0["fold"]==fold][train_col]

df_train_y = df0[df0["fold"]!=fold]["next water level"]
df_valid_y = df0[df0["fold"]==fold]["next water level"]

# validだけ変化
print(df_train_x.shape)
print(df_valid_x.shape)

(6688214, 43)
(1398876, 43)


In [12]:
import lightgbm as lgb

# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 4000, early_stopping_rounds = 200, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.508930 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8191
[LightGBM] [Info] Number of data points in the train set: 6688214, number of used features: 43
[LightGBM] [Info] Start training from score 2.391359
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.118651
[200]	valid_0's rmse: 0.118051
[300]	valid_0's rmse: 0.117857
[400]	valid_0's rmse: 0.117739
[500]	valid_0's rmse: 0.11786
[600]	valid_0's rmse: 0.117909
Early stopping, best iteration is:
[451]	valid_0's rmse: 0.117697


In [13]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.1176965459339264


In [14]:
importance = model.feature_importance()

# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 821
hour_cos: 733
station_c: 3244
h-0: 244
h-6: 110
h-12: 204
h-15: 119
h-18: 143
h-21: 181
h-22: 197
h-23: 615
mean: 52
max: 115
min: 169
std: 587
nullcount: 60
0-12mean: 35
12-24mean: 65
0-6mean: 49
6-12mean: 36
12-18mean: 59
18-24mean: 157
std_1d_ago: 491
nullcount_1d_ago: 84
0-6mean_1d_ago: 166
6-12mean_1d_ago: 128
12-18mean_1d_ago: 127
18-24mean_1d_ago: 164
std_2d_ago: 395
nullcount_2d_ago: 68
0-12mean_2d_ago: 244
12-24mean_2d_ago: 84
12-14-mean_rain: 236
12-14-max_rain: 116
15-17-mean_rain: 293
15-17-max_rain: 99
18-20-mean_rain: 269
18-20-max_rain: 119
21-23-mean_rain: 694
21-23-max_rain: 270
mean_rain: 623
mean_1d_ago_rain: 441
mean_2d_ago_rain: 424


In [15]:
import pickle

file = '../model/model_20230102-2-159-fold1.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=2

In [16]:
fold=2
# 5年、1年
df_train_x = df0[df0["fold"]!=fold][train_col]
df_valid_x = df0[df0["fold"]==fold][train_col]

df_train_y = df0[df0["fold"]!=fold]["next water level"]
df_valid_y = df0[df0["fold"]==fold]["next water level"]

# validだけ変化
print(df_train_x.shape)
print(df_valid_x.shape)

(6692440, 43)
(1394650, 43)


In [17]:
import lightgbm as lgb

# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 4000, early_stopping_rounds = 200, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.730416 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8197
[LightGBM] [Info] Number of data points in the train set: 6692440, number of used features: 43
[LightGBM] [Info] Start training from score 2.376687
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.111519
[200]	valid_0's rmse: 0.111019
[300]	valid_0's rmse: 0.110293
[400]	valid_0's rmse: 0.109864
[500]	valid_0's rmse: 0.109797
[600]	valid_0's rmse: 0.109664
[700]	valid_0's rmse: 0.109613
[800]	valid_0's rmse: 0.109544
[900]	valid_0's rmse: 0.109561
[1000]	valid_0's rmse: 0.109501
[1100]	valid_0's rmse: 0.109424
[1200]	valid_0's rmse: 0.109396
[1300]	valid_0's rmse: 0.109389
[1400]	valid_0's rmse: 0.109274
[1500]	valid_0's rmse: 0.109283
[1600]	valid_0's rmse: 0.10925
[1700]	valid_0's rmse: 0.109293
Early stopping, best iteration is:
[1555]	valid_0's rmse:

In [18]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.10923125783081729


In [19]:
importance = model.feature_importance()

# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 3312
hour_cos: 2608
station_c: 9162
h-0: 1382
h-6: 534
h-12: 868
h-15: 683
h-18: 674
h-21: 718
h-22: 746
h-23: 1517
mean: 197
max: 462
min: 443
std: 2439
nullcount: 160
0-12mean: 147
12-24mean: 256
0-6mean: 169
6-12mean: 247
12-18mean: 279
18-24mean: 491
std_1d_ago: 1904
nullcount_1d_ago: 320
0-6mean_1d_ago: 607
6-12mean_1d_ago: 441
12-18mean_1d_ago: 462
18-24mean_1d_ago: 488
std_2d_ago: 2007
nullcount_2d_ago: 299
0-12mean_2d_ago: 897
12-24mean_2d_ago: 400
12-14-mean_rain: 1171
12-14-max_rain: 451
15-17-mean_rain: 841
15-17-max_rain: 417
18-20-mean_rain: 878
18-20-max_rain: 409
21-23-mean_rain: 1680
21-23-max_rain: 661
mean_rain: 1984
mean_1d_ago_rain: 1336
mean_2d_ago_rain: 1503


In [20]:
import pickle

file = '../model/model_20230102-2-159-fold2.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=3

In [6]:
fold=3
# 5年、1年
df_train_x = df0[df0["fold"]!=fold][train_col]
df_valid_x = df0[df0["fold"]==fold][train_col]

df_train_y = df0[df0["fold"]!=fold]["next water level"]
df_valid_y = df0[df0["fold"]==fold]["next water level"]

# validだけ変化
print(df_train_x.shape)
print(df_valid_x.shape)

(6997335, 43)
(1089755, 43)


In [8]:
import lightgbm as lgb

# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 4000, early_stopping_rounds = 300, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.707797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8218
[LightGBM] [Info] Number of data points in the train set: 6997335, number of used features: 43
[LightGBM] [Info] Start training from score 2.379709
Training until validation scores don't improve for 300 rounds
[100]	valid_0's rmse: 0.142186
[200]	valid_0's rmse: 0.14134
[300]	valid_0's rmse: 0.142543
[400]	valid_0's rmse: 0.143595
[500]	valid_0's rmse: 0.142723
Early stopping, best iteration is:
[250]	valid_0's rmse: 0.140926


In [9]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.14092633504002522


In [10]:
importance = model.feature_importance()

# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 496
hour_cos: 405
station_c: 1686
h-0: 137
h-6: 53
h-12: 81
h-15: 57
h-18: 71
h-21: 103
h-22: 119
h-23: 458
mean: 20
max: 83
min: 154
std: 328
nullcount: 17
0-12mean: 10
12-24mean: 29
0-6mean: 18
6-12mean: 16
12-18mean: 15
18-24mean: 88
std_1d_ago: 202
nullcount_1d_ago: 47
0-6mean_1d_ago: 46
6-12mean_1d_ago: 66
12-18mean_1d_ago: 45
18-24mean_1d_ago: 11
std_2d_ago: 232
nullcount_2d_ago: 49
0-12mean_2d_ago: 102
12-24mean_2d_ago: 37
12-14-mean_rain: 173
12-14-max_rain: 88
15-17-mean_rain: 179
15-17-max_rain: 76
18-20-mean_rain: 202
18-20-max_rain: 70
21-23-mean_rain: 515
21-23-max_rain: 169
mean_rain: 312
mean_1d_ago_rain: 202
mean_2d_ago_rain: 233


In [11]:
import pickle

file = '../model/model_20230102-2-159-fold3.pkl'
pickle.dump(model, open(file, 'wb'))

#### k=4

In [12]:
fold=4
# 5年、1年
df_train_x = df0[df0["fold"]!=fold][train_col]
df_valid_x = df0[df0["fold"]==fold][train_col]

df_train_y = df0[df0["fold"]!=fold]["next water level"]
df_valid_y = df0[df0["fold"]==fold]["next water level"]

# validだけ変化
print(df_train_x.shape)
print(df_valid_x.shape)

(6698700, 43)
(1388390, 43)


In [13]:
import lightgbm as lgb

# データセットを生成する
lgb_train = lgb.Dataset(df_train_x, df_train_y)
lgb_eval = lgb.Dataset(df_valid_x, df_valid_y, reference=lgb_train)

# LightGBM のハイパーパラメータ
lgbm_params = {
    'objective': 'regression',
    'metric':'rmse',
    'seed': 42,
    
}

# 上記のパラメータでモデルを学習する
model = lgb.train(params=lgbm_params, num_boost_round = 4000, early_stopping_rounds = 200, train_set=lgb_train, valid_sets=lgb_eval, categorical_feature = categorical_col, verbose_eval = 100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.600809 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8198
[LightGBM] [Info] Number of data points in the train set: 6698700, number of used features: 43
[LightGBM] [Info] Start training from score 2.379827
Training until validation scores don't improve for 200 rounds
[100]	valid_0's rmse: 0.0991359
[200]	valid_0's rmse: 0.0984829
[300]	valid_0's rmse: 0.0984847
[400]	valid_0's rmse: 0.0987923
Early stopping, best iteration is:
[292]	valid_0's rmse: 0.0984196


In [14]:
# テストデータを予測する
y_pred = model.predict(df_valid_x, num_iteration=model.best_iteration)
# compute RMSE
print('\nRMSE:', np.sqrt(((df_valid_y-y_pred)**2).mean()))


RMSE: 0.09841964379282693


In [15]:
importance = model.feature_importance()

# 各特徴量の重要度を表示
for i, val in enumerate(importance):
    print(f"{df_train_x.columns[i]}: {val}")

hour_sin: 524
hour_cos: 429
station_c: 2100
h-0: 175
h-6: 70
h-12: 90
h-15: 60
h-18: 67
h-21: 110
h-22: 84
h-23: 492
mean: 24
max: 107
min: 138
std: 406
nullcount: 21
0-12mean: 10
12-24mean: 31
0-6mean: 12
6-12mean: 16
12-18mean: 26
18-24mean: 92
std_1d_ago: 272
nullcount_1d_ago: 62
0-6mean_1d_ago: 106
6-12mean_1d_ago: 73
12-18mean_1d_ago: 58
18-24mean_1d_ago: 76
std_2d_ago: 259
nullcount_2d_ago: 49
0-12mean_2d_ago: 114
12-24mean_2d_ago: 32
12-14-mean_rain: 257
12-14-max_rain: 119
15-17-mean_rain: 180
15-17-max_rain: 89
18-20-mean_rain: 217
18-20-max_rain: 69
21-23-mean_rain: 507
21-23-max_rain: 185
mean_rain: 415
mean_1d_ago_rain: 274
mean_2d_ago_rain: 263


In [16]:
import pickle

file = '../model/model_20230102-2-159-fold4.pkl'
pickle.dump(model, open(file, 'wb'))

RMSE: 0.10429121969099113 => RMSE: 0.1050101947431999  
RMSE: 0.08597841291308117 => RMSE: 0.08721021824930242   
-----
RMSE: 0.0990825958936637 => RMSE: 0.09879354007639315    
RMSE: 0.08675756093517713 => RMSE: 0.08671721421971205  
-----  
RMSE: 0.10746136800577111 => RMSE: 0.10938408575628061  
RMSE: 0.08676155107886897 => RMSE: 0.08615205182499786  
-----
RMSE: 0.09169186183425276 => RMSE: 0.09179547691883928   
RMSE: 0.0857240846895323 => RMSE: 0.08535579774614441 